# Overhead of using a generator

## Plain function calls

In [ ]:
def test(a):
    return a

%timeit test(1)

So we get 1E7 function calls per second (a few thousand CPU clock cycles). This is about 100 times slower than what can be achieved in low level languages.
CPython does use the C stack for function calls and spins up a new interpreter loop for each call. However, it uses a custom frame object for the actual Python content.

## Plain list

In [ ]:
numbers = list(range(1000))

%timeit sum(numbers)

## Generator

In [16]:
def generator_func():
    i = 0
    while i < 1000:
        i += 1
        yield i

%timeit sum(generator_func())

10000 loops, best of 3: 106 µs per loop


So while this is ten times slower, we can still manage about 1E7 iterations per second (similar to the number of function calls we can manage).
Under the hood CPython keeps the frame object and reuses it when entering the function again, but a new C stack frame is pushed each time.

What about nesting generators?

In [17]:
def generator_func():
    i = 0
    while i < 1000:
        i += 1
        yield i
        
def generator_wrapper(inner_iterator):
    yield from inner_iterator

%timeit sum(generator_wrapper(generator_wrapper(generator_func())))

10000 loops, best of 3: 199 µs per loop


So while nesting added some overhead, the effect is not dramatic (we still manage about five million iterations per second).

## Plain function calls in a generator

In [15]:
def test_func(i):
    return i + 1;

def generator_calling_func():
    i = 0
    while i < 1000:
        i = test_func(i)
        yield i
        
%timeit sum(generator_calling_func())

10000 loops, best of 3: 194 µs per loop


# asyncio overhead

The cost of switching to another thread can be estimated at $> 30 \mu s$. If the estimate entering a generator $100ns$ the this is enough for 300 such entering, so one would assume that the async/await approach has at least 10 times less overhead. 

In [6]:
import asyncio
loop = asyncio.get_event_loop()

In [14]:
async def counter():
    sum = 0
    i = 0
    while i < 1000:
        i = await get_next(i)
        sum += i
    return sum

async def get_next(i):
    return i + 1
    
%timeit loop.run_until_complete(counter())

1000 loops, best of 3: 454 µs per loop


So each iteration takes about $400ns$, and the event loop could perform about 2 million iterations per second.